In [8]:

import matplotlib.pyplot as plt
# import squarify   # pip install squarify (algorithm for treemap)
import pandas as pd 
import numpy as np

data_dir = '../Data/Clinical/'
cl1 = pd.read_csv( data_dir + "clinical_data_1.csv")
cl2 = pd.read_csv( data_dir + "clinical_data_2.csv")
# all = pd.read_csv( data_dir + "clinical_data.csv")

bcrid1 = []
bcrid2 = []
data_out = '../Result/'
with open(data_out + "BCR1_ID_list.txt", "r") as file:
    for line in file:
        bcrid1.append(line.strip().strip('"'))
with open(data_out + "BCR2_ID_list.txt", "r") as file:
    for line in file:
        bcrid2.append(line.strip().strip('"'))

len(cl1), len(cl2), len(all)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 10: invalid start byte

In [20]:
cl1.head()

,CMD_ID,ID,PNTTM,SEX,CMD_BIR,AGE,CMD_DATE1,CMD_DATE2,CMD_GROUP,CMD_ADDRESS,...,CMD_LACTATER,CMD_TROPONIN,CMD_TROPONINR,CMD_FERRITIN,CMD_FERRITINR,CMD_NOTE__2,CMD_ER,CMD_EDATE,CMD_RINFO,CMD_RDATE
0,1.0,COV-CCO-001,1,1,1959.2.5,62,2021.1.12,2021.1.12,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666
1,2.0,COV-CCO-001,2,1,1959.2.5,62,2021.1.13,2021.1.13,3.0,6.0,...,3.0,1.0,2.4,1.0,230.0,77777,66666.0,66666,66666.0,66666
2,3.0,COV-CCO-001,3,1,1959.2.5,62,2021.1.14,2021.1.14,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666
3,4.0,COV-CCO-001,4,1,1959.2.5,62,2021.1.15,2021.1.15,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666
4,5.0,COV-CCO-001,5,1,1959.2.5,62,2021.1.16,2021.1.16,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666


In [2]:
colist01 = cl1.columns
colist02 = cl2.columns

both = list(set(colist01) & set(colist02))
colist1 = list(set(colist01) - set(colist02))
colist2 = list(set(colist02) - set(colist01))

print(both)
print("Both:", len(both))
print("Only in colist1:", len(colist1))
print("Only in colist2:", len(colist2))

['SEX', 'PNTTM', 'ID', 'AGE']
Both: 4
Only in colist1: 208
Only in colist2: 318


In [5]:
both_idlist = []
idlist01 = [ x for x in cl1['ID'].unique() if x not in cl2['ID']]
idlist02 = [ x for x in cl2['ID'].unique() if x not in cl1['ID']]

for x in cl1['ID']:
    for y in cl2['ID']:
        if x == y:
            both_idlist.append(x)
        else:
            break
        


print("Both:", len(both_idlist))
print("Only in cl1:", len(idlist01))
print("Only in cl2:", len(idlist02))

Both: 0
Only in cl1: 420
Only in cl2: 239


In [4]:
cl1['PNTTM'].unique(), cl2['PNTTM'].unique()

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       dtype=int64),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int64))

In [6]:
# 각 부분의 개수를 저장할 딕셔너리를 초기화합니다.

def count_substrings(idlist):
    counts = {"CCO": 0, "SCO": 0, "CNO": 0, "MCO": 0, "CNC": 0}
    for id_str in idlist:
        substring = id_str[4:7]
        if substring in counts:
            counts[substring] += 1

    return counts

cl1_counts = count_substrings(idlist01)
cl2_counts = count_substrings(idlist02)

for key in cl1_counts:
    print(key, cl1_counts[key])
    
print("--------------------")

for key in cl2_counts:
    print(key, cl2_counts[key])

CCO 229
SCO 67
CNO 81
MCO 4
CNC 39
--------------------
CCO 127
SCO 26
CNO 0
MCO 6
CNC 80


In [7]:
df = pd.DataFrame(cl1)
df = df[df['PNTTM']==7]

cno_cnc_df = df[df['ID'].str.contains('CNO|CNC')]
cno_df = df[df['ID'].str.contains('CNO')]
cnc_df = df[df['ID'].str.contains('CNC')]

shared_pattern = {}
cno_pattern = {}
cnc_pattern = {}
for col in cno_cnc_df.columns[1:]:
    values = cno_cnc_df[col].unique()
    values02 = cno_df[col].unique()
    values03 = cnc_df[col].unique()
    if len(values) == 1:
        shared_pattern[col] = values[0]
        if values02[0] != values03[0]:
            cno_pattern[col] = values02[0]
            cnc_pattern[col] = values03[0]

print("CNO|CNC common patterns:")
print(shared_pattern)
print("CNO common patterns:")
print(cno_pattern)
print("CNC common patterns:")
print(cnc_pattern)

CNO|CNC common patterns:
{'PNTTM': 2, 'CMD_GROUP': 1.0, 'CMD_COVID19': 2.0, 'CMD_DISEASE': 77777.0, 'CMD_DISEASEEXP': 77777.0, 'CMD_HOS': 2.0, 'CMD_CDATE': '77777', 'CMD_PATH': 77777.0, 'CMD_PATHEXP': '77777', 'CMD_FEV': 2.0, 'CMD_COUGH': 2.0, 'CMD_SPUTUM': 2.0, 'CMD_BREATH': 2.0, 'CMD_THROAT': 2.0, 'CMD_RHINO': 2.0, 'CMD_MUSCLE': 2.0, 'CMD_FATIGUE': 2.0, 'CMD_HEADACHE': 2.0, 'CMD_CONFU': 2.0, 'CMD_VOMIT': 2.0, 'CMD_DIARRHEA': 2.0, 'CMD_ETC': 2.0, 'CMD_ETCEXP': '77777', 'CMD_DDATE': '77777', 'CMD_HEARTF': 2.0, 'CMD_CPOD': 2.0, 'CMD_TUBER': 2.0, 'CMD_ASTHMA': 2.0, 'CMD_CKIDNEY': 2.0, 'CMD_CKIDNEYP': 2.0, 'CMD_NEOPLASM': 2.0, 'CMD_HIV': 2.0, 'CMD_HIV2': 2.0, 'CMD_AVIRUS__1': 2.0, 'CMD_AVIRUSR__1': 2.0, 'CMD_AVIRUSO__1': 2.0, 'CMD_AVIRUSOEXP__1': 77777.0, 'CMD_ABIOTICM__1': 2.0, 'CMD_ABIOTICT__1': 2.0, 'CMD_ABIOTICQ__1': 2.0, 'CMD_ABIOTICG__1': 2.0, 'CMD_ABIOTICB__1': 2.0, 'CMD_STEROID__1': 2.0, 'CMD_STEROIDD__1': 2.0, 'CMD_STEROIDM__1': 2.0, 'CMD_STEROIDP__1': 2.0, 'CMD_STEROIDC__1': 2.0

'CMD_GROUP': 1.0 (대상자 그룹 (1:정상인, 2:non-COVID, 3:COVID))

'CMD_COVID19': 2.0 (COVID검사 결과(1:양성, 2:음성))

In [8]:
def extract_common_pattern(df, keyword):
    pattern = {}
    keyword_df = df[df['ID'].str.contains(keyword)]
    for col in keyword_df.columns[1:]:
        values = keyword_df[col].unique()
        values = [val for val in values if (not isinstance(val, str) or len(val) <= 4) and (not isinstance(val, (int, float)) or int(val) < 1000)]
        if len(values) == 1:
            pattern[col] = values[0]
    return pattern

def print_common_patterns(df, keywords):
    for keyword in keywords:
        keyword_pattern = extract_common_pattern(df, keyword)
        print(f"{keyword} common patterns:")
        print(keyword_pattern)

**df02 = pd.DataFrame(cl2)**

**df02 = df02[df02['PNTTM'] == 2]**

여기서, cl1 혹은 cl2와 (각각 1,2차 수집) PNMT == 1 ~ 30 혹은 1 ~ 27 (관찰횟수) 조정하면서 확인 할 것 


In [12]:
df02 = pd.DataFrame(cl2)
df02 = df02[df02['PNTTM'] == 3]

keywords = ['CCO', 'SCO', 'MCO', 'CNC', 'CNO']
print_common_patterns(df02, keywords)

CCO common patterns:
{'PNTTM': 3, 'M1_PREG': 77777, 'M1_COVID19': 77777, 'M1_VACN4': 77777, 'M1_CKIDNEYP': 77777, 'M1_HIV2': 77777, 'M1_AVIRUSR': 77777, 'M1_AVIRUSO': 77777, 'M1_AVIRUSOEXP': 77777, 'M1_ABIOTICM': 77777, 'M1_ABIOTICT': 77777, 'M1_ABIOTICQ': 77777, 'M1_ABIOTICG': 77777, 'M1_ABIOTICB': 77777, 'M1_ABIOTICO': 77777, 'M1_STEROIDD': 77777, 'M1_STEROIDM': 77777, 'M1_STEROIDP': 77777, 'M1_STEROIDC': 77777, 'M1_HEIGHT': 173.7, 'M1_WEIGHT': 107.5, 'M1_BMI': 35.6, 'M1_TEMP': 37.3, 'M1_NOTE': '없음', 'M2_NIV': 2, 'M2_NIVO': 77777, 'M2_NOGAS': 2, 'M2_CRRT': 2, 'M2_PRONE': 2, 'M2_PLASMA': 2, 'M2_PYRAMAX': 2, 'M2_MK4482': 2, 'M2_VASOPRESSOR': 2, 'M2_DVT': 2, 'M2_SHOCK': 2, 'M2_BACTEREMIA': 2, 'M2_CAA': 2, 'M2_CAM': 2, 'M2_CAAR': 2, 'M2_ARI': 2, 'M2_ETCCOMPEXP': '뇌졸중', 'M2_NOTE': '없음', 'M3_ER': 77777, 'M3_RINFO': 77777, 'M4_HRATE': 77777, 'M4_BRATE': 77777, 'M4_SYSBP': 77777, 'M4_RELBP': 77777, 'M4_BLOODT': 77777, 'M4_HAEMOGLOBIN': 77777, 'M4_WBC': 77777, 'M4_WBCC': 77777, 'M4_NEUTROPHIL

In [39]:
df02 = pd.DataFrame(cl2)
# df02 = df02[df02['PNTTM'] == 1]
df02 = df02[df02['ID'].str.contains('CCO')]

print(df02['M1_GROUP'].unique())
# print(sum(df02['M1_COVID19'] == 2))

[    2 77777 99999]


In [12]:
print(bcrid1)
print(bcrid2)
print(idlist01)
print(idlist02)

['COV-CCO-041', 'COV-CCO-042', 'COV-CCO-043', 'COV-CCO-044', 'COV-CCO-045', 'COV-CCO-046', 'COV-CCO-048', 'COV-CCO-050', 'COV-CCO-051', 'COV-CCO-052', 'COV-CCO-053', 'COV-CCO-054', 'COV-CCO-055', 'COV-CCO-056', 'COV-CCO-057', 'COV-CCO-058', 'COV-CCO-059', 'COV-CCO-060', 'COV-CCO-061', 'COV-CCO-062', 'COV-CCO-063', 'COV-CCO-064', 'COV-CCO-065', 'COV-CCO-066', 'COV-CCO-067', 'COV-CCO-068', 'COV-CCO-069', 'COV-CCO-070', 'COV-CCO-073', 'COV-CCO-074', 'COV-CCO-075', 'COV-CCO-076', 'COV-CCO-077', 'COV-CCO-078', 'COV-CCO-081', 'COV-CCO-082', 'COV-CCO-083', 'COV-CCO-084', 'COV-CCO-085', 'COV-CCO-086', 'COV-CCO-121', 'COV-CCO-122', 'COV-CCO-124', 'COV-CCO-125', 'COV-CCO-126', 'COV-CCO-127', 'COV-CCO-129', 'COV-CCO-130', 'COV-CCO-132', 'COV-CCO-133', 'COV-CCO-134', 'COV-CCO-135', 'COV-CCO-136', 'COV-CCO-137', 'COV-CCO-138', 'COV-CCO-139', 'COV-CCO-140', 'COV-CCO-141', 'COV-CCO-142', 'COV-CCO-143', 'COV-CCO-144', 'COV-CCO-146', 'COV-CCO-147', 'COV-CCO-151', 'COV-CCO-152', 'COV-CCO-153', 'COV-CCO-

In [26]:
trim01 = cl1[cl1['ID'].isin(bcrid1)]
trim02 = cl2[cl2['ID'].isin(bcrid2)]

datalist = [trim01, trim02]
for i in datalist:
    display(i.head(), i.shape)

,CMD_ID,ID,PNTTM,SEX,CMD_BIR,AGE,CMD_DATE1,CMD_DATE2,CMD_GROUP,CMD_ADDRESS,...,CMD_LACTATER,CMD_TROPONIN,CMD_TROPONINR,CMD_FERRITIN,CMD_FERRITINR,CMD_NOTE__2,CMD_ER,CMD_EDATE,CMD_RINFO,CMD_RDATE
484,484.0,COV-CCO-041,1,1,1990.6.30,31,2021.2.6,2021.2.6,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666
485,485.0,COV-CCO-041,2,1,1990.6.30,31,2021.2.7,2021.2.7,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666
486,486.0,COV-CCO-041,3,1,1990.6.30,31,2021.2.8,2021.2.8,3.0,6.0,...,2.3,1.0,3.6,1.0,251.0,77777,66666.0,66666,66666.0,66666
487,487.0,COV-CCO-041,4,1,1990.6.30,31,2021.2.9,2021.2.9,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666
488,488.0,COV-CCO-041,5,1,1990.6.30,31,2021.2.10,2021.2.10,3.0,6.0,...,77777.0,2.0,77777.0,2.0,77777.0,77777,66666.0,66666,66666.0,66666


(1871, 212)

,ID,PNTTM,SEX,REG_BIR,AGE,REG_VISIT,REG_DATE,M1_GROUP,M1_ADDRESS,M1_PREG,...,M4_FIVCRES,M4_DLCOND,M4_DLCO,M4_DLCORES,M4_LAST,M4_SURV,M4_DD,M4_DRTYPE,FIN_D,FIN_DDATE
33,COV-CCO-239,1,1,1951-06-01,70,내원/입원 시,2021-06-02,2,6,77777,...,77777,77777,77777.0,77777,77777,77777,77777,77777,77777,77777
34,COV-CCO-239,2,1,1951-06-01,70,입원 2일차,2021-06-03,77777,77777,77777,...,77777,77777,77777.0,77777,77777,77777,77777,77777,77777,77777
35,COV-CCO-239,3,1,1951-06-01,70,입원 3일차,2021-06-04,77777,77777,77777,...,77777,77777,77777.0,77777,77777,77777,77777,77777,77777,77777
36,COV-CCO-239,4,1,1951-06-01,70,입원 4일차,2021-06-05,77777,77777,77777,...,77777,77777,77777.0,77777,77777,77777,77777,77777,77777,77777
37,COV-CCO-239,5,1,1951-06-01,70,입원 5일차,2021-06-06,77777,77777,77777,...,77777,77777,77777.0,77777,77777,77777,77777,77777,77777,77777


(554, 322)